# Домашнее задание №1: линейная регрессия (10 баллов).

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

#### Напишите сюда свой вариант: 3


In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=1337)

У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_train, y_train)
print(mean_squared_error(y_test, reg.predict(X_test)))

2.3479427223196442e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [170]:
from sklearn.linear_model import SGDRegressor
reg_sgd = SGDRegressor(alpha=1e-15).fit(X_train, y_train)
print(mean_squared_error(y_test, reg_sgd.predict(X_test)))
reg_sgd.coef_

2.6547360356996457e-10


array([ 6.81118488e+01,  5.67527667e+00, -8.35167982e-07, -3.32867378e-07,
       -2.78065121e-07,  6.92378200e-07, -9.58721608e-07,  6.50213361e-07,
        1.47613532e-06,  1.33472433e-06, -2.18038216e-06,  5.04446692e-07,
       -4.10664980e-07,  1.40009224e-06, -8.50566469e-07, -3.99961021e-07,
       -1.07904758e-06,  9.20855869e+01,  6.55280005e+01,  5.80255519e-07,
        1.10383518e-07,  1.55268484e-07,  1.17040809e-07, -1.37322697e-06,
        7.51070887e-07, -9.27848448e-07,  1.43019224e-06,  2.07070380e-06,
       -1.98133798e-06, -9.30261090e-07, -4.81945394e-06, -4.18835305e-07,
       -5.99349079e-07,  5.46281837e+01, -3.32393490e-06, -1.79885292e-06,
       -1.13637556e-06,  5.84001657e+01,  1.52491238e-06, -3.45610686e-07,
       -2.21500776e-06,  3.40203824e-07,  9.19122358e-07, -1.57305345e-06,
       -4.79378197e-07,  9.17819169e+00, -5.76264891e-07, -3.19130331e-06,
        1.24278893e-06, -2.79236012e-06, -2.29551087e-06, -8.65197996e-07,
       -3.67843111e-07, -

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

Метод градиентного спуска показывает себя хуже(но не сильно), потому что из-за инциализации рандомной точки мы можем попасть в локальный, а не глобальный минимум. При обучении метода линейной регрессии мы обучаем модель так, чтобы она сразу минимизировала среднеквадратическую ошибку

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию.

In [132]:

class sLinearRegression(object):
    def __init__(self, alpha=0.01, l_ratio=0.00001, tol=1e-4, max_iter=1000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.weights, self.bias = None, None

    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''

        # Инициализируем веса значениями
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        for i in range(self.max_iter):
            # Рассчитываем предсказания
            predictions = np.dot(X, self.weights) + self.bias

            # Вычисляем ошибку
            error = predictions - y

            # Рассчитываем градиент с L2-регуляризацией
            gradient = (2 / X.shape[0]) * np.dot(X.T, error) + 2 * self.l_ratio * self.weights
            gradient_bias = (2 / X.shape[0]) * np.sum(error)

            # Обновляем веса с использованием градиентного спуска
            self.weights -= self.alpha * gradient
            self.bias -= self.alpha * gradient_bias

            # Проверяем критерий останова
            if np.linalg.norm(gradient) < self.tol:
                break

    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''

        return np.dot(X, self.weights) + self.bias


In [174]:
type(my_reg.predict(X_test))

numpy.ndarray

In [172]:
mean_squared_error(y_test, my_reg.predict(X_test))

4.356061707440644e-06

In [171]:
from sklearn.metrics import mean_squared_error
my_reg = sLinearRegression(alpha=0.1, max_iter=10000)
my_reg.fit(X_train, y_train)
assert mean_squared_error(y_test, my_reg.predict(X_test)) < 1e-3
print('You are amazing! Great work!')

You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [175]:
#your code here
from sklearn.linear_model import Ridge

r_lr = Ridge(alpha=0.1)
r_lr.fit(X_train, y_train)
print(f'MSE для Ridge', mean_squared_error(y_test, r_lr.predict(X_test)))
print(f'MSE для собственной регрессии', mean_squared_error(y_test, my_reg2.predict(X_test)))



MSE для Ridge 0.00010602236561537249
MSE для собственной регрессии 342.80785239530763


In [166]:
my_reg2 = sLinearRegression(alpha=0.01, max_iter=10000, l_ratio=0.1)
my_reg2.fit(X_train, y_train)
print(mean_squared_error(y_test, my_reg2.predict(X_test)))
print('You are amazing! Great work!')

342.80785239530763
You are amazing! Great work!
